In [5]:
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

from functools import partial
from tqdm import tqdm, trange
tqdm.pandas(ncols=100, mininterval=1)
tqdm, trange = partial(tqdm, ncols=100, mininterval=1), partial(trange, ncols=100, mininterval=1)

In [6]:
%time textual_novelty_metrics = pd.read_csv("../../20230910/extra/textual_novelty/papers_textual_metrics.csv")

CPU times: user 57 s, sys: 10.4 s, total: 1min 7s
Wall time: 1min 7s


In [7]:
%%time
openalex_papers = pd.read_parquet("/kellogg/proj/dashun/Sciscinet-v2/sciscinet_papers.parquet")
openalex_papers['doi'] = openalex_papers['doi'].str.lower().str.replace('https://doi.org/', '', regex=False)

CPU times: user 3min 58s, sys: 1min 2s, total: 5min
Wall time: 3min 13s


In [8]:
openalex_paper_id_to_doi = openalex_papers.set_index("paperid").doi.to_dict()
textual_novelty_metrics['doi'] = textual_novelty_metrics.PaperID.progress_map(lambda x: openalex_paper_id_to_doi.get(x, None))

textual_novelty_metrics_2 = textual_novelty_metrics[["PaperID", "doi", "semantic_distance"]]
textual_novelty_metrics_2.columns = ["openalex_paper_id", "doi", "semantic_distance"]

100%|███████████████████████████████████████████████| 75295921/75295921 [01:36<00:00, 777913.55it/s]


In [ ]:
dimensions_papers = pd.read_parquet("../parquet/processed/publications.parquet")
dimensions_papers['doi'] = dimensions_papers.doi.str.lower().str.replace('https://doi.org/', '', regex=False)
dimensions_papers = dimensions_papers.dropna(subset=["doi"])
doi_to_dim_paper_id = dimensions_papers.set_index("doi").id.to_dict()

In [23]:
textual_novelty_metrics_2['paper_id'] = textual_novelty_metrics_2.doi.progress_map(lambda x: doi_to_dim_paper_id.get(x, None))

100%|███████████████████████████████████████████████| 75295921/75295921 [01:44<00:00, 721750.78it/s]
/tmp/ipykernel_849984/1628914978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textual_novelty_metrics_2['paper_id'] = textual_novelty_metrics_2.doi.progress_map(lambda x: doi_to_dim_paper_id.get(x, None))


In [24]:
textual_novelty_metrics_3 = textual_novelty_metrics_2[["paper_id", "openalex_paper_id", "doi", "semantic_distance"]]

In [25]:
print(f"# original papers with textual novelty metrics = {textual_novelty_metrics.PaperID.notna().sum()}")
print(f"# final papers with OpenAlex IDs = {textual_novelty_metrics_3.openalex_paper_id.notna().sum()}")
print(f"# final papers with DOIs = {textual_novelty_metrics_3.doi.notna().sum()}")
print(f"# final papers with Dimensions IDs = {textual_novelty_metrics_3.paper_id.notna().sum()}")

# original papers with textual novelty metrics = 75295921
# final papers with OpenAlex IDs = 75295921
# final papers with DOIs = 66335795
# final papers with Dimensions IDs = 65551453


In [26]:
textual_novelty_metrics_3.to_parquet("data/textual_novelty.parquet", index=False)